In [1]:
"""Apply Fishers exact test, odds ratio and Fst statistics on multiallelic variations """

#  Used modules

import pandas as pd 
import numpy as np 
import scipy 



In [2]:
# Loading files


#  Fst statistics files

Fst_exones = pd.read_excel("FstExones_Case_vs_Control.xlsx").fillna("NaN")
Fst_genes = pd.read_excel("FstGenes_Case_vs_Control.xlsx").fillna("NaN")


#  Files - no empty genotypes in all subject; filtered by Cadd_phred > 15

main_exones_multi = pd.read_excel("Exones_variance_pdfilter_allelesdone_multi.xlsx")
main_genes_multi = pd.read_excel("Genes_variance_pdfilter_allelesdone_multi.xlsx")

In [3]:

#  Copy against fragmentation

## Main files

main_exones_multi_copy = main_exones_multi.copy()
main_genes_multi_copy = main_genes_multi.copy()

##  Fst files

Fst_exones_copy = Fst_exones.copy()
Fst_genes_copy = Fst_genes.copy()



In [4]:
#  Creating double CN_alleles

##  Exones

main_exones_multi_copy["CN_2O"] = main_exones_multi_copy["CN_O"] * 2
main_exones_multi_copy["CN_2R"] = main_exones_multi_copy["CN_R"] * 2

##  Genes

main_genes_multi_copy["CN_2O"] = main_genes_multi_copy["CN_O"] * 2
main_genes_multi_copy["CN_2R"] = main_genes_multi_copy["CN_R"] * 2



### Extracting Fst data according chromosom and position from used files

In [5]:
#  Creating Fst column for Fst scores data

#  Combining CHROM|POS - Combined columns

##  Exones

Fst_exones_copy["Combined_Fst"] = Fst_exones_copy["CHROM"].astype(str) + "_" + Fst_exones_copy["POS"].astype(str)
main_exones_multi_copy["Combined_main_exones"] = main_exones_multi_copy["CHROM"].astype(str) + "_" + main_exones_multi_copy["POS"].astype(str)

##  Genes

Fst_genes_copy["Combined_Fst"] = Fst_genes_copy["CHROM"].astype(str) + "_" + Fst_genes_copy["POS"].astype(str)
main_genes_multi_copy["Combined_main_genes"] = main_genes_multi_copy["CHROM"].astype(str) + "_" + main_genes_multi_copy["POS"].astype(str)

#  Merging dataframes → setting similar combined data

##  Exones

merged_exones = Fst_exones_copy.merge(main_exones_multi_copy, left_on='Combined_Fst', right_on='Combined_main_exones', how='left')

##  Genes

merged_genes = Fst_genes_copy.merge(main_genes_multi_copy,left_on='Combined_Fst', right_on='Combined_main_genes', how='left')


#  Filtering only usefull columns from merged dataframes

##  Exones

merged_exones_position = merged_exones.loc[:,["CHROM_x", "POS_x", "WEIR_AND_COCKERHAM_FST", "Combined_Fst", "CHROM_y", "POS_y", "Combined_main_exones"]].reset_index(drop=True)

##  Genes

merged_genes_position = merged_genes.loc[:,["CHROM_x", "POS_x", "WEIR_AND_COCKERHAM_FST", "Combined_Fst", "CHROM_y", "POS_y", "Combined_main_genes"]].reset_index(drop=True)


#  Setting Fst columns

##  Exones

merged_exones_filtered = merged_exones_position[merged_exones_position["Combined_main_exones"].notna()].reset_index(drop=True)

merged_exones_filtered_dict = dict(zip(merged_exones_filtered["Combined_main_exones"],merged_exones_filtered["WEIR_AND_COCKERHAM_FST"]))

main_exones_multi_copy["Fst"] = main_exones_multi_copy["Combined_main_exones"].map(merged_exones_filtered_dict).fillna("NaN")

##  Genes

merged_genes_filtered = merged_genes_position[merged_genes_position["Combined_main_genes"].notna()].reset_index(drop=True)

merged_genes_filtered_dict = dict(zip(merged_genes_filtered["Combined_main_genes"],merged_genes_filtered["WEIR_AND_COCKERHAM_FST"]))

main_genes_multi_copy["Fst"] = main_genes_multi_copy["Combined_main_genes"].map(merged_genes_filtered_dict).fillna("NaN")

### Application of fishers exact test (scipy.stats)


In [6]:
#  FET

##  Exones

FET1_exones = main_exones_multi_copy.apply(lambda x: scipy.stats.fisher_exact([[x.CS_O,x.CS_R],[x.CN_O,x.CN_R]]), axis=1)
FET2_exones = main_exones_multi_copy.apply(lambda x: scipy.stats.fisher_exact([[x.CS_O,x.CS_R],[x.CN_2O, x.CN_2R]]), axis=1)

##  Genes

FET1_genes = main_genes_multi_copy.apply(lambda x: scipy.stats.fisher_exact([[x.CS_O,x.CS_R],[x.CN_O,x.CN_R]]), axis=1)
FET2_genes = main_genes_multi_copy.apply(lambda x: scipy.stats.fisher_exact([[x.CS_O,x.CS_R],[x.CN_2O, x.CN_2R]]), axis=1)


### Application Odds ratio

- scipy.stats.contingency.odds_ratio
- output:
- confidence interval - only for more details
- odds ratio - main result


In [7]:
#  OR

## Exones

###  OR1


OR1_exones = main_exones_multi_copy.apply(lambda x: scipy.stats.contingency.odds_ratio([[x.CS_O, x.CS_R],[x.CN_O, x.CN_R]], kind='sample'), axis=1)  #  Applying OR

OR1_exones_results = OR1_exones.apply(lambda res: res.statistic) #  Getting result


###  OR2

OR2_exones = main_exones_multi_copy.apply(lambda x: scipy.stats.contingency.odds_ratio([[x.CS_O, x.CS_R],[x.CN_2O, x.CN_2R]], kind='sample'), axis=1)  #  Applying OR

OR2_exones_results = OR2_exones.apply(lambda res: res.statistic) #  Getting result

#  Genes

#  OR1

OR1_genes = main_genes_multi_copy.apply(lambda x: scipy.stats.contingency.odds_ratio([[x.CS_O, x.CS_R],[x.CN_O, x.CN_R]], kind='sample'), axis=1)  #  Applying OR

OR1_genes_results = OR1_genes.apply(lambda res: res.statistic)  #  Getting result


#  OR2

OR2_genes = main_genes_multi_copy.apply(lambda x: scipy.stats.contingency.odds_ratio([[x.CS_O, x.CS_R],[x.CN_2O, x.CN_2R]], kind='sample'), axis=1)  #  Applying OR

OR2_genes_results = OR2_genes.apply(lambda res: res.statistic)  #  Getting result


In [8]:
#  Adding confidence interval

#  Exones

OR1_exones_CI = OR1_exones.apply(lambda res: res.confidence_interval(confidence_level=0.95))

OR2_exones_CI = OR2_exones.apply(lambda res: res.confidence_interval(confidence_level=0.95))

#  Genes

OR1_genes_CI = OR1_genes.apply(lambda res: res.confidence_interval(confidence_level=0.95))

OR2_genes_CI = OR2_genes.apply(lambda res: res.confidence_interval(confidence_level=0.95))


c:\Users\chumi\Documents\Programming_space\.venv\Lib\site-packages\scipy\stats\_odds_ratio.py:156: RuntimeWarning: divide by zero encountered in divide
  se = np.sqrt((1/table).sum())
c:\Users\chumi\Documents\Programming_space\.venv\Lib\site-packages\scipy\stats\_odds_ratio.py:168: RuntimeWarning: invalid value encountered in scalar subtract
  loglow = log_or - z*se


In [9]:
#  Extraction FET p-values, Odds ratio and Confidence interval (low, high)

#  Exones

#  P-values

main_exones_multi_copy["FET1"] = FET1_exones.apply(lambda x: x[1]).fillna("NaN")
main_exones_multi_copy["FET2"] = FET2_exones.apply(lambda x: x[1]).fillna("NaN")

#  OR

main_exones_multi_copy["OR1"] = OR1_exones_results.fillna("NaN")
main_exones_multi_copy["OR2"] = OR2_exones_results.fillna("NaN")


#  Confidmulty_ence interval

#  Regular CN counts group - interval - <low,high>

main_exones_multi_copy["OR1_low"] = OR1_exones_CI.apply(lambda res: res.low).fillna("NaN")
main_exones_multi_copy["OR1_high"] = OR1_exones_CI.apply(lambda res: res.high).fillna("NaN")


#  Doubmulty_le CN counts group - interval - <low, high>

main_exones_multi_copy["OR2_low"] = OR2_exones_CI.apply(lambda res: res.low).fillna("NaN")
main_exones_multi_copy["OR2_high"] = OR2_exones_CI.apply(lambda res: res.high).fillna("NaN")


#  Genemulty_s

#  P-values

main_genes_multi_copy["FET1"] = FET1_genes.apply(lambda x: x[1]).fillna("NaN")
main_genes_multi_copy["FET2"] = FET2_genes.apply(lambda x: x[1]).fillna("NaN")

#  OR

main_genes_multi_copy["OR1"] = OR1_genes_results.fillna("NaN")
main_genes_multi_copy["OR2"] = OR2_genes_results.fillna("NaN")

#  Confidence interval

#  Regular CN counts group - interval - <low,high>

main_genes_multi_copy["OR1_low"] = OR1_genes_CI.apply(lambda res: res.low).fillna("NaN")
main_genes_multi_copy["OR1_high"] = OR1_genes_CI.apply(lambda res: res.high).fillna("NaN")


#  Double CN counts group - interval - <low, high>

main_genes_multi_copy["OR2_low"] = OR2_genes_CI.apply(lambda res: res.low).fillna("NaN")
main_genes_multi_copy["OR2_high"] = OR2_genes_CI.apply(lambda res: res.high).fillna("NaN")

### Formatting and export final output file

In [13]:
# Final files - stats_exon; main_genes_multi_copy

# necessary columns per 

# poziční - CHROM, POS, REF, ALT, ID
# FORMAT - data experimentálních subjektů |
# OR - odds ratio | dle počtu CN
# Fisher exact test - FET  | dle počtu CN


# CN - regular

col_stats_1 = ["CHROM","POS","REF","ALT","ID","GENE","FEATURE_ID","HGVS_c","HGVS_p","EFFECT","IMPACT","CS_AA","CS_AR","CS_RR","CS_RO","CS_OO","CN_AA","CN_AR","CN_RR","CN_RO","CN_OO","CS_O","CS_R","CN_O","CN_R","CN_2O","CN_2R","FET1","OR1","OR1_low","OR1_high","Fst"]


# CN - double

col_stats_2 = ["CHROM","POS","REF","ALT","ID","GENE","FEATURE_ID","HGVS_c","HGVS_p","EFFECT","IMPACT","CS_AA","CS_AR","CS_RR","CS_RO","CS_OO","CN_AA","CN_AR","CN_RR","CN_RO","CN_OO","CS_O","CS_R","CN_O","CN_R","CN_2O","CN_2R","FET1","FET2","OR1","OR1_low","OR1_high","OR2","OR2_low","OR2_high","Fst"]

# main_exones_multi_copy


# exones - FET1

stats_exones_FET1 = main_exones_multi_copy.loc[:,col_stats_1]

# exones - FET2

stats_exones_FET2 = main_exones_multi_copy.loc[:,col_stats_2]


# main_genes_multi_copy


# genes - FET1

stats_genes_FET1 = main_genes_multi_copy.loc[:,col_stats_1]

# genes - FET2

stats_genes_FET2 = main_genes_multi_copy.loc[:,col_stats_2]

In [14]:
# Final statistical output |

# stats_exones_FET

stats_exones_FET1.to_excel("StatsExones_FET1_multi.xlsx", index=False)
stats_exones_FET2.to_excel("StatsExones_FET2_multi.xlsx", index=False)

# stats_genes_FET

stats_genes_FET1.to_excel("StatsGenes_FET1_multi.xlsx", index=False)
stats_genes_FET2.to_excel("StatsGenes_FET2_multi.xlsx", index=False)